In this notebook, we will be building our model for the open-world experiments to binary classify by determining whether the web traffic trace corresponds to a monitored website or unmonitored website.   

In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import psutil
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Extracting Data

To begin, we will first import the dataframes into this notebook. Run either 1 of these blocks

If you are using Google Colab, run this block

In [ ]:
# 1 Use this with colab
print("Loading datafile...")
with open('datasets/extracted_features.pkl', 'rb') as f:
    extracted_df = pickle.load(f)
print ("Data loaded")

If you are using local, run this block

In [ ]:
# 2 Use this for local (change the directory to where the extracted_features.pkl is stored on your local machine)
# Load the pickle file
print("Loading datafile...")
# change this directory to the directory where mon_standard.pkl is stored on your local machine
file_path = r'C:\EWHA\Term 2\Machine Learning\pro\neurotic_networkers\extracted_features.pkl' # Jordans local path
with open(file_path, 'rb') as f: # Path to extracted_features.pkl in Colab
    extracted_df = pickle.load(f)
print ("Data loaded")

In [ ]:
extracted_df

We relabel the data such that monitored websites will be labelled as 1 and unmonitored websites will be labelled as 0

In [ ]:
relabelled_df = extracted_df.copy()
relabelled_df['label'] = relabelled_df['label'].apply(lambda x: 1 if x>=0 else x)

In [ ]:
relabelled_df

In [ ]:
X_initial = relabelled_df.drop(columns=['label'])
y_initial = relabelled_df['label']

# Constructing Model with all features

Here, we construct a model using all of the features.

In [ ]:
# Split data into training and testing sets
X_initial_train, X_initial_test, y_initial_train, y_initial_test = train_test_split(
    X_initial, y_initial, test_size=0.2, random_state=42
)

In [ ]:
# Initialize the model
clf_all_features = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=10, min_samples_split=2, max_features="sqrt", random_state=42)

# Train the model
clf_all_features.fit(X_initial_train, y_initial_train)

In [ ]:
# Create a process object to track current process
process = psutil.Process(os.getpid())
# tracking start time
start_time = time.time()
# Make predictions on the train set
y_initial_train_pred = clf_all_features.predict(X_initial_train)
# tracking end time
end_time = time.time()

time_taken_all_features_train = end_time - start_time
memory_used_all_features_train = process.memory_info().rss / (1024 * 1024)

In [ ]:
# Print accuracy and other metrics
print("Memory used:", memory_used_all_features_train, "MB")
print("Time taken to predict:", time_taken_all_features_train, "seconds")
print("Model Accuracy:", accuracy_score(y_initial_train, y_initial_train_pred))
print(classification_report(y_initial_train, y_initial_train_pred))
print("Confusion Matrix")
print(confusion_matrix(y_initial_train, y_initial_train_pred))

In [ ]:
# Create a process object to track current process
process = psutil.Process(os.getpid())
# tracking start time
start_time = time.time()
# Make predictions on the test set
y_initial_test_pred = clf_all_features.predict(X_initial_test)
# tracking end time
end_time = time.time()

time_taken_all_features_test = end_time - start_time
memory_used_all_features_test = process.memory_info().rss / (1024 * 1024)

In [ ]:
# Print accuracy and other metrics
print("Memory used:", memory_used_all_features_test, "MB")
print("Time taken to predict:", time_taken_all_features_test, "seconds")
print("Model Accuracy:", accuracy_score(y_initial_test, y_initial_test_pred))
print(classification_report(y_initial_test, y_initial_test_pred))
print("Confusion Matrix")
print(confusion_matrix(y_initial_test, y_initial_test_pred))

In [ ]:
classes = list(set(y_initial_train)) 
y_test_bin = label_binarize(y_initial_train, classes=classes)

# Get predicted probabilities
y_pred_proba = clf_all_features.predict_proba(X_initial_train)

# Plot ROC curve
plt.figure(figsize=(15, 10))

# Store individual class ROC curves
fpr_dict = {}
tpr_dict = {}
roc_auc_dict = {}

# Compute ROC curve for each class
for i, class_label in enumerate(classes):
    fpr_dict[i], tpr_dict[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc_dict[i] = auc(fpr_dict[i], tpr_dict[i])
    
    plt.plot(
        fpr_dict[i], tpr_dict[i], 
        label=f'Class {class_label} (AUC = {roc_auc_dict[i]:.2f})'
    )

# Compute Macro Average ROC Curve
# First, create a grid of FPR points
all_fpr = np.unique(np.concatenate([fpr_dict[i] for i in range(len(classes))]))

# Interpolate ROC curves
mean_tpr = np.zeros_like(all_fpr)
for i in range(len(classes)):
    mean_tpr += np.interp(all_fpr, fpr_dict[i], tpr_dict[i])

# Average the TPR
mean_tpr /= len(classes)

# Compute macro-average AUC
macro_roc_auc = auc(all_fpr, mean_tpr)

# Plot macro-average ROC curve
plt.plot(
    all_fpr, mean_tpr, 
    color='black', 
    linestyle='--', 
    linewidth=2, 
    label=f'Macro-average (AUC = {macro_roc_auc:.2f})'
)

# Add plot details
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
plt.title('ROC Curve for Website Classification', fontsize=16)
plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=14)
plt.ylabel('True Positive Rate (Sensitivity)', fontsize=14)
plt.legend(loc='lower right', fontsize=12)
plt.grid(alpha=0.3)
plt.show()

# Optional: Print macro-average AUC
print(f"Macro-average AUC: {macro_roc_auc:.2f}")

In [ ]:
classification_report_dict = classification_report(y_initial_test, y_initial_test_pred)
df = pd.DataFrame(classification_report_dict).T 
df = df.drop('support', axis=1)  
df = df.drop('accuracy', axis=0, errors='ignore')  
sns.heatmap(df, annot=True, cmap="YlGnBu", fmt='.2f')
plt.set_title('Heatmap for Classification Report')
plt.set_ylabel('Classes')
plt.set_xlabel('Metrics')
plt.show()

# Feature Importance

In [ ]:
df_X = pd.DataFrame(X_initial_train)

In [ ]:
df_X.head()

In [ ]:
# using entropy
model_1= RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=10, min_samples_split=2, max_features="sqrt", random_state=42)
model_1.fit(X_initial_train, y_initial_train)
feature_imp_1 = pd.Series(model_1.feature_importances_, index=df_X.columns)

In [ ]:
# using gini
model_2= RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=10, min_samples_split=2, max_features="sqrt", random_state=42)
model_2.fit(X_initial_train, y_initial_train)
feature_imp_2 = pd.Series(model_2.feature_importances_, index=df_X.columns)

In [ ]:
feature_names = df_X.columns
entropy_values = feature_imp_1.values
gini_values = feature_imp_2.values

bar_width = 0.4 
y_positions = range(len(feature_names)) 

plt.figure(figsize=(10,6))

#plotting entropy importance
plt.barh(
    [y - bar_width / 2 for y in y_positions],
    entropy_values,
    bar_width,
    label='Entropy',
    color='blue',
)

# plotting gini improtance
plt.barh(
    [y + bar_width / 2 for y in y_positions],
    gini_values,
    bar_width,
    label='Gini',
    color='orange',
)

plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.yticks(y_positions, feature_names) 
plt.title('Visualisation of Feature Importance: Entropy vs Gini')
plt.legend()
plt.tight_layout()
plt.show()

Recall the extracted features:
> **Feature Group 1: Traffic Volume (Absolute)**  
> - Feature 1: Number of incoming packets  
> - Feature 2: Number of outgoing packets  
> - Feature 3: Total number of packets  
> 
> **Feature Group 2: Traffic Volume (Fraction)**
> - Feature 1: Number of incoming packets as a fraction of the total number of packets  
> - Feature 2: Number of outgoing packets as a fraction of the total number of packets 
> 
> **Feature Group 3: Traffic Ordering List**
> - Feature 6: Standard deviation of the outgoing packets ordering list  
> - Feature 7: Average of the outgoing packets ordering list  
> 
> **Feature Group 4: Traffic concentration** 
> - Feature 8: Sum of all items in the alternative concentration feature list  
> - Feature 9: Average of all items in the alternative concentration feature list  

We noted that within each of the 4 feature groups, the features are likely to be highly correlated due to their similarity. Furthermore, feature groups 1 and 2 are closely related as well with 1 being an absolute measurement of traffic volume and 2 as the ratio. Hence we will be selecting 2 features from the combination of group 1 and 2, and 1 feature each from group 3 and 4.

According to our feature importance analysis, we have selected the features to be
1. Feature 1: Number of incoming packets   
2. Feature 3: Total number of packets
3. Feature 7: Average of the outgoing packets ordering list 
4. Feature 8: Sum of all items in the alternative concentration feature list  

In [ ]:
X = relabelled_df[['incoming_packet_counts', 'total_packet_counts', 'avg_outgoing_order', 'sum_concentration']]
y = relabelled_df['label']

# Constructing Model with selected features

In this section, we construct an arbitrary random forest classification model using arbitrarily chosen parameters. This section aims to explore the implementation of the model. These parameters will be tuned in the next section. 

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
df = pd.DataFrame(X_train)

In [ ]:
df.head()

In [ ]:
# Initialize the model
clf_selected_features = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=10, min_samples_split=2, max_features="sqrt", random_state=42)

# Train the model
clf_selected_features.fit(X_train, y_train)

In [ ]:
# Create a process object to track current process
process = psutil.Process(os.getpid())
# tracking start time
start_time = time.time()
# Make predictions on the train set
y_train_pred = clf_selected_features.predict(X_train)
# tracking end time
end_time = time.time()


time_taken_selected_features_train = end_time - start_time
memory_used_selected_features_train = process.memory_info().rss / (1024 * 1024)

In [ ]:
# Print accuracy and other metrics
print("Memory used:", memory_used_selected_features_train, "MB")
print("Time taken to predict:", time_taken_selected_features_train, "seconds")
print("Model Accuracy:", accuracy_score(y_train, y_train_pred))
print(classification_report(y_train, y_train_pred))
print("Confusion Matrix")
print(confusion_matrix(y_train, y_train_pred))

In [ ]:
# Create a process object to track current process
process = psutil.Process(os.getpid())
# tracking start time
start_time = time.time()
# Make predictions on the test set
y_test_pred = clf_selected_features.predict(X_test)
# tracking end time
end_time = time.time()
# Make probability predictions on the test set
y_test_pred_proba = clf_selected_features.predict_proba(X_test)

time_taken_selected_features_test = end_time - start_time
memory_used_selected_features_tuned_test = process.memory_info().rss / (1024 * 1024)

In [ ]:
# Print accuracy and other metrics
print("Memory used:", memory_used_selected_features_tuned_test, "MB")
print("Time taken to predict:", time_taken_selected_features_test, "seconds")
print("Model Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))
print("Confusion Matrix")
print(confusion_matrix(y_test, y_test_pred))


# Model Tuning

In this section, we will be using Grid Search to tune our model parameters for our Random Forest Classifier

In [ ]:
# Defining the parameter grid for the Grid Search
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [10, 15],
    'min_samples_leaf': [5, 10],
    'bootstrap': [True],
    'criterion': ['gini', 'entropy']           
}

# Defining the grid search
grid = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5,  
    refit=True, 
    verbose = 3,
    n_jobs=-1
)

In [ ]:
# Fitting the grid search
start_time = time.time()
grid.fit(X_initial_train, y_initial_train)
end_time = time.time()
time_taken_grid_search = end_time - start_time

In [ ]:
# Print the time taken to perform the grid search
print("Time taken for grid search:", time_taken_grid_search, "seconds")

In [ ]:
# Print best parameters after grid search
print("Best parameters found:", grid.best_params_)

In [ ]:
# Create a process object to track current process
process = psutil.Process(os.getpid())
# tracking start time
start_time = time.time()
# Make predictions on the test set
y_initial_train_pred_tuned = grid.predict(X_initial_train)
# tracking end time
end_time = time.time()

time_taken_selected_features_tuned_train = end_time - start_time
memory_used_selected_features_tuned_train = process.memory_info().rss / (1024 * 1024)

In [ ]:
# Print accuracy and other metrics
print("Memory used:", memory_used_selected_features_tuned_train, "MB")
print("Time taken to predict:", time_taken_selected_features_tuned_train, "seconds")
print("Model Accuracy:", accuracy_score(y_initial_train, y_initial_train_pred_tuned))
print(classification_report(y_initial_train, y_initial_train_pred_tuned))
print("Confusion Matrix")
print(confusion_matrix(y_initial_train, y_initial_train_pred_tuned))

In [ ]:
# Create a process object to track current process
process = psutil.Process(os.getpid())
# tracking start time
start_time = time.time()
# Make predictions on the test set
y_initial_test_pred_tuned = grid.predict(X_initial_test)
# tracking end time
end_time = time.time()

time_taken_selected_features_tuned_test = end_time - start_time
memory_used_selected_features_tuned_test = process.memory_info().rss / (1024 * 1024)

In [ ]:
# Print accuracy and other metrics
print("Memory used:", memory_used_selected_features_tuned_test, "MB")
print("Time taken to predict:", time_taken_selected_features_tuned_test, "seconds")
print("Model Accuracy:", accuracy_score(y_initial_test, y_initial_test_pred_tuned))
print(classification_report(y_initial_test, y_initial_test_pred_tuned))
print("Confusion Matrix")
print(confusion_matrix(y_initial_test, y_initial_test_pred_tuned))

In [ ]:
from sklearn.metrics import roc_curve, auc
from scipy.interpolate import interp1d
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import numpy as np

# Binarize the test labels
classes = grid.classes_
y_test_bin = label_binarize(y_initial_test, classes=classes)

# Get predicted probabilities
y_test_proba = grid.predict_proba(X_initial_test)

# Function to smooth the ROC curve
def smooth_curve(x, y, num_points=3):
    # Ensure unique x values
    unique_x, indices = np.unique(x, return_index=True)
    unique_y = y[indices]
    
    # Interpolate
    f = interp1d(unique_x, unique_y, kind='quadratic', fill_value="extrapolate")
    x_new = np.linspace(0, 1, num_points)
    y_new = f(x_new)
    
    return x_new, y_new

# Plot ROC curve with smoothing
plt.figure(figsize=(12, 8))
roc_aucs = []

# Compute ROC for each class
for i, class_label in enumerate(classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_test_proba[:, i])
    roc_auc = auc(fpr, tpr)
    
    # Smooth the curve
    fpr_smooth, tpr_smooth = smooth_curve(fpr, tpr)
    
    # Store for macro-average calculation
    roc_aucs.append((class_label, roc_auc, fpr_smooth, tpr_smooth))
    
    # Plot smoothed curve for each class
    plt.plot(fpr_smooth, tpr_smooth, label=f'Class {class_label} (AUC = {roc_auc:.2f})')

# Macro-average ROC curve
try:
    # Prepare for macro-average
    all_fpr = np.unique(np.concatenate([roc[2] for roc in roc_aucs]))
    mean_tpr = np.zeros_like(all_fpr)
    
    # Interpolate and average TPR
    for _, _, fpr, tpr in roc_aucs:
        mean_tpr += np.interp(all_fpr, fpr, tpr)
    mean_tpr /= len(roc_aucs)
    
    # Compute macro-average AUC
    macro_auc = auc(all_fpr, mean_tpr)
    
    # Plot macro-average ROC curve
    plt.plot(all_fpr, mean_tpr, label=f'Macro-average (AUC = {macro_auc:.2f})', 
             color='black', linestyle='--', linewidth=2)
except Exception as e:
    print(f"Error computing macro-average: {e}")

# Plot chance line
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')

# Customize the plot
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Smoothed Receiver Operating Characteristic (ROC) Curve', fontsize=14)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
classification_report_dict = classification_report(y_initial_test, y_initial_test_pred_tuned)
df = pd.DataFrame(classification_report_dict).T 
df = df.drop('support', axis=1)  
df = df.drop('accuracy', axis=0, errors='ignore')  
sns.heatmap(df, annot=True, cmap="YlGnBu", fmt='.2f')
plt.set_title('Heatmap for Classification Report')
plt.set_ylabel('Classes')
plt.set_xlabel('Metrics')
plt.show()